<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия № 2
Автор материала: программист-исследователь Mail.ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ Юрий Кашницкий. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Домашнее задание № 10
## <center> Прогнозирование задержек вылетов

Ваша задача – побить как минимум 2 бенчмарка в [соревновании](https://www.kaggle.com/c/flight-delays-2017) на Kaggle Inclass. Подробных инструкций не будет, будет только тезисно описано, как получен второй – с помощью Xgboost. Надеюсь, на данном этапе курса вам достаточно бросить полтора взгляда на данные, чтоб понять, что это тот тип задачи, в которой затащит градиентный бустинг. Скорее всего Xgboost, но тут у нас еще немало категориальных признаков...

<img src='../../img/xgboost_meme.jpg' width=40% />

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

C:\Users\user\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train = pd.read_csv('D:/flight_delays_train.csv')
test = pd.read_csv('D:/flight_delays_test.csv')

In [3]:
train.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [4]:
test.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-7,c-25,c-3,615,YV,MRY,PHX,598
1,c-4,c-17,c-2,739,WN,LAS,HOU,1235
2,c-12,c-2,c-7,651,MQ,GSP,ORD,577
3,c-3,c-25,c-7,1614,WN,BWI,MHT,377
4,c-6,c-6,c-3,1505,UA,ORD,STL,258


Итак, надо по времени вылета самолета, коду авиакомпании-перевозчика, месту вылета и прилета и расстоянию между аэропортами вылета и прилета предсказать задержку вылета более 15 минут. В качестве простейшего бенчмарка возьмем логистическую регрессию и два признака, которые проще всего взять: `DepTime` и `Distance`. У такой модели результат – 0.68202 на LB. 

In [5]:
X_train, y_train = train[['Distance', 'DepTime']].values, train['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values
X_test = test[['Distance', 'DepTime']].values

X_train_part, X_valid, y_train_part, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=17)

scaler = StandardScaler()
X_train_part = scaler.fit_transform(X_train_part)
X_valid = scaler.transform(X_valid)

C:\Users\user\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [6]:
logit = LogisticRegression()

logit.fit(X_train_part, y_train_part)
logit_valid_pred = logit.predict_proba(X_valid)[:, 1]

roc_auc_score(y_valid, logit_valid_pred)

0.67956914653526068

In [7]:
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

logit.fit(X_train_scaled, y_train)
logit_test_pred = logit.predict_proba(X_test_scaled)[:, 1]

pd.Series(logit_test_pred, name='dep_delayed_15min').to_csv('logit_2feat.csv', index_label='id', header=True)

C:\Users\user\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


Второй бенчмарк, представленный в рейтинге соревнования, был получен так:
- Признаки `Distance` и  `DepTime` брались без изменений
- Создан признак "маршрут" из исходных `Origin` и `Dest`
- К признакам `Month`, `DayofMonth`, `DayOfWeek`, `UniqueCarrier` и "маршрут" применено OHE-преобразование (`LabelBinarizer`)
- Выделена отложенная выборка
- Обучалась логистическая регрессия и градиентный бустинг (xgboost), гиперпараметры бустинга настраивались на кросс-валидации, сначала те, что отвечают за сложность модели, затем число деревьев фиксировалось равным 500 и настраивался шаг градиентного спуска
- С помощью `cross_val_predict` делались прогнозы обеих моделей на кросс-валидации (именно предсказанные вероятности), настраивалась линейная смесь ответов логистической регрессии и градиентного бустинга вида $w_1 * p_{logit} + (1 - w_1) * p_{xgb}$, где $p_{logit}$ – предсказанные логистической регрессией вероятности класса 1, $p_{xgb}$ – аналогично. Вес $w_1$ подбирался вручную. 
- В качестве ответа для тестовой выборки бралась аналогичная комбинация ответов двух моделей, но уже обученных на всей обучающей выборке.

Описанный план ни к чему не обязывает – это просто то, как решение получил автор задания. Возможно, вы не захотите следовать намеченному плану, а добавите, скажем, пару хороших признаков и обучите лес из тысячи деревьев.

Удачи!

In [8]:
train['route'] = train.Origin + '-' + train.Dest
import sklearn.preprocessing


In [40]:
test['route'] = test.Origin + '-' + test.Dest


In [45]:
lbmon = sklearn.preprocessing.LabelBinarizer()
mymonth = lbmon.fit_transform(train.Month)
mymonth = pd.DataFrame(mymonth)
#mymonth.head()
mymonthtest = pd.DataFrame(lbmon.transform(test.Month))
mymonth.head(), mymonthtest.head()

(   0   1   2   3   4   5   6   7   8   9   10  11
 0   0   0   0   0   0   0   0   0   0   0   1   0
 1   0   0   0   0   0   0   1   0   0   0   0   0
 2   0   0   0   0   0   0   0   0   0   0   0   1
 3   0   0   1   0   0   0   0   0   0   0   0   0
 4   0   1   0   0   0   0   0   0   0   0   0   0,
    0   1   2   3   4   5   6   7   8   9   10  11
 0   0   0   0   0   0   0   0   0   0   1   0   0
 1   0   0   0   0   0   0   1   0   0   0   0   0
 2   0   0   0   1   0   0   0   0   0   0   0   0
 3   0   0   0   0   0   1   0   0   0   0   0   0
 4   0   0   0   0   0   0   0   0   1   0   0   0)

In [46]:
lbdom = sklearn.preprocessing.LabelBinarizer()
mydayofmonth = lbdom.fit_transform(train.DayofMonth)
mydayofmonth = pd.DataFrame(mydayofmonth)
mydayofmonthtest = pd.DataFrame(lbdom.transform(test.DayofMonth))
mydayofmonth.head(), mydayofmonthtest.head()

(   0   1   2   3   4   5   6   7   8   9  ...  21  22  23  24  25  26  27  28  \
 0   0   0   0   0   0   0   0   0   0   0 ...   0   0   0   0   0   0   0   0   
 1   0   0   0   0   0   0   0   0   0   0 ...   0   0   0   0   0   0   0   0   
 2   0   0   0   0   0   0   0   0   0   0 ...   0   0   0   0   0   0   0   0   
 3   0   0   0   0   0   0   0   0   0   0 ...   0   0   0   0   0   0   0   0   
 4   0   0   0   0   0   0   0   0   0   0 ...   0   0   0   0   0   0   0   1   
 
    29  30  
 0   0   0  
 1   0   0  
 2   0   0  
 3   0   0  
 4   0   0  
 
 [5 rows x 31 columns],
    0   1   2   3   4   5   6   7   8   9  ...  21  22  23  24  25  26  27  28  \
 0   0   0   0   0   0   0   0   0   0   0 ...   0   0   0   0   0   0   0   0   
 1   0   0   0   0   0   0   0   0   1   0 ...   0   0   0   0   0   0   0   0   
 2   0   0   0   0   0   0   0   0   0   0 ...   0   0   0   0   0   0   0   0   
 3   0   0   0   0   0   0   0   0   0   0 ...   0   0   0   0   0   0   0

In [47]:
lbweek = sklearn.preprocessing.LabelBinarizer()
mydayofweek = lbweek.fit_transform(train.DayOfWeek)
mydayofweek = pd.DataFrame(mydayofweek)
mydayofweektest = pd.DataFrame(lbweek.transform(test.DayOfWeek))
mydayofweek.head(), mydayofweektest.head()

(   0  1  2  3  4  5  6
 0  0  0  0  0  0  0  1
 1  0  0  1  0  0  0  0
 2  0  0  0  0  1  0  0
 3  0  0  0  0  0  1  0
 4  0  0  0  0  0  1  0,    0  1  2  3  4  5  6
 0  0  0  1  0  0  0  0
 1  0  1  0  0  0  0  0
 2  0  0  0  0  0  0  1
 3  0  0  0  0  0  0  1
 4  0  0  1  0  0  0  0)

In [48]:
lbuc = sklearn.preprocessing.LabelBinarizer()
myuc = lbuc.fit_transform(train.UniqueCarrier)
myuc = pd.DataFrame(myuc)
myuctest = pd.DataFrame(lbuc.transform(test.UniqueCarrier))
myuc.head(), myuctest.head()

(   0   1   2   3   4   5   6   7   8   9  ...  12  13  14  15  16  17  18  19  \
 0   1   0   0   0   0   0   0   0   0   0 ...   0   0   0   0   0   0   0   0   
 1   0   0   0   0   0   0   0   0   0   0 ...   0   0   0   0   0   0   1   0   
 2   0   0   0   0   0   0   0   0   0   0 ...   0   0   0   0   0   0   0   0   
 3   0   0   0   0   0   0   0   0   0   0 ...   0   0   0   1   0   0   0   0   
 4   0   0   0   0   0   0   0   0   0   0 ...   0   0   0   0   0   0   0   1   
 
    20  21  
 0   0   0  
 1   0   0  
 2   1   0  
 3   0   0  
 4   0   0  
 
 [5 rows x 22 columns],
    0   1   2   3   4   5   6   7   8   9  ...  12  13  14  15  16  17  18  19  \
 0   0   0   0   0   0   0   0   0   0   0 ...   0   0   0   0   0   0   0   0   
 1   0   0   0   0   0   0   0   0   0   0 ...   0   0   0   0   0   0   0   1   
 2   0   0   0   0   0   0   0   0   0   0 ...   1   0   0   0   0   0   0   0   
 3   0   0   0   0   0   0   0   0   0   0 ...   0   0   0   0   0   0   0

In [49]:
lbroute = sklearn.preprocessing.LabelBinarizer()
myroute = lbroute.fit_transform(train.route)
myroute = pd.DataFrame(myroute)
myroutetest = pd.DataFrame(lbroute.transform(test.route))
myroute.head(), myroutetest.head()

(   0     1     2     3     4     5     6     7     8     9     ...   4419  \
 0     0     0     0     0     0     0     0     0     0     0  ...      0   
 1     0     0     0     0     0     0     0     0     0     0  ...      0   
 2     0     0     0     0     0     0     0     0     0     0  ...      0   
 3     0     0     0     0     0     0     0     0     0     0  ...      0   
 4     0     0     0     0     0     0     0     0     0     0  ...      0   
 
    4420  4421  4422  4423  4424  4425  4426  4427  4428  
 0     0     0     0     0     0     0     0     0     0  
 1     0     0     0     0     0     0     0     0     0  
 2     0     0     0     0     0     0     0     0     0  
 3     0     0     0     0     0     0     0     0     0  
 4     0     0     0     0     0     0     0     0     0  
 
 [5 rows x 4429 columns],
    0     1     2     3     4     5     6     7     8     9     ...   4419  \
 0     0     0     0     0     0     0     0     0     0     0  ...   

In [14]:
XX_train = np.concatenate((X_train, mymonth, mydayofmonth, mydayofweek, myuc, myroute), axis=1)

In [50]:
XXX_train = np.concatenate((X_train_scaled, mymonth, mydayofmonth, mydayofweek, myuc, myroute), axis=1)

In [51]:
XXX_test = np.concatenate((X_test_scaled, mymonthtest, mydayofmonthtest, mydayofweektest, myuctest, myroutetest), axis=1)

In [16]:
XX_train.shape, X_train.shape

((100000, 4503), (100000, 2))

In [17]:
XX_train_part, XX_valid, yy_train_part, yy_valid = train_test_split(XX_train, y_train, test_size=0.3, random_state=17)


In [52]:
XXX_train_part, XXX_valid, yyy_train_part, yyy_valid = train_test_split(XXX_train, y_train, test_size=0.3, random_state=17)


In [19]:
scaler = StandardScaler()
XX_train_part = scaler.fit_transform(XX_train_part)
XX_valid = scaler.transform(XX_valid)

C:\Users\user\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [20]:
logit = LogisticRegression()

logit.fit(XX_train_part, yy_train_part)
logit_valid_pred = logit.predict_proba(XX_valid)[:, 1]

roc_auc_score(yy_valid, logit_valid_pred)

0.66682696714059098

In [53]:
logit = LogisticRegression()

logit.fit(XXX_train_part, yyy_train_part)
logit_valid_pred = logit.predict_proba(XXX_valid)[:, 1]

'new', roc_auc_score(yyy_valid, logit_valid_pred)

('new', 0.69086566209268929)

In [55]:
logit.fit(XXX_train, y_train)
logit_test_pred = logit.predict_proba(XXX_test)[:, 1]

pd.Series(logit_test_pred, name='dep_delayed_15min').to_csv('D:/logit_manyfeats.csv', index_label='id', header=True)

In [22]:
XXX_train_part_df = pd.DataFrame(XX_train_part)

In [23]:
import xgboost as xgb

In [24]:
xgb1 = XGBClassifier(learning_rate=0.1)

In [25]:
xgb1.fit(XXX_train_part, yyy_train_part)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [26]:
xgb_valid_pred = xgb1.predict_proba(XXX_valid)[:, 1]

roc_auc_score(yyy_valid, xgb_valid_pred)

0.71193476392608246

In [27]:
xgb1 = XGBClassifier(learning_rate=0.3)
xgb1.fit(XXX_train_part, yyy_train_part)
xgb_valid_pred = xgb1.predict_proba(XXX_valid)[:, 1]

roc_auc_score(yyy_valid, xgb_valid_pred)

0.71675439778423811

In [30]:
xgb1 = XGBClassifier(learning_rate=0.5)
xgb1.fit(XXX_train_part, yyy_train_part)
xgb_valid_pred = xgb1.predict_proba(XXX_valid)[:, 1]

roc_auc_score(yyy_valid, xgb_valid_pred)

0.71950880898810388

In [32]:
xgb1 = XGBClassifier(learning_rate=0.5, max_depth=5)
xgb1.fit(XXX_train_part, yyy_train_part)
xgb_valid_pred = xgb1.predict_proba(XXX_valid)[:, 1]

roc_auc_score(yyy_valid, xgb_valid_pred)

0.72102673570641918

In [33]:
xgb1 = XGBClassifier(learning_rate=0.3, max_depth=5)
xgb1.fit(XXX_train_part, yyy_train_part)
xgb_valid_pred = xgb1.predict_proba(XXX_valid)[:, 1]

roc_auc_score(yyy_valid, xgb_valid_pred)

0.72027534174260399

In [36]:
xgb1 = XGBClassifier(learning_rate=0.3, max_depth=7)
xgb1.fit(XXX_train_part, yyy_train_part)
xgb_valid_pred = xgb1.predict_proba(XXX_valid)[:, 1]

roc_auc_score(yyy_valid, xgb_valid_pred)

0.72353921278651134

In [37]:
xgb1 = XGBClassifier(learning_rate=0.3, max_depth=10)
xgb1.fit(XXX_train_part, yyy_train_part)
xgb_valid_pred = xgb1.predict_proba(XXX_valid)[:, 1]

roc_auc_score(yyy_valid, xgb_valid_pred)

0.72355595467865341

In [38]:
xgb1 = XGBClassifier(learning_rate=0.3, max_depth=10, n_estimators=500)
xgb1.fit(XXX_train_part, yyy_train_part)
xgb_valid_pred = xgb1.predict_proba(XXX_valid)[:, 1]

roc_auc_score(yyy_valid, xgb_valid_pred)

0.72531983415516454

In [39]:
xgb1 = XGBClassifier(learning_rate=0.1, max_depth=10, n_estimators=500)
xgb1.fit(XXX_train_part, yyy_train_part)
xgb_valid_pred = xgb1.predict_proba(XXX_valid)[:, 1]

roc_auc_score(yyy_valid, xgb_valid_pred)

0.72918549933074139

In [67]:
xgb1 = XGBClassifier(learning_rate=0.1, max_depth=10, n_estimators=700)
xgb1.fit(XXX_train_part, yyy_train_part)
xgb_valid_pred = xgb1.predict_proba(XXX_valid)[:, 1]

roc_auc_score(yyy_valid, xgb_valid_pred)

0.72993790520701496

In [68]:
xgb1 = XGBClassifier(learning_rate=0.1, max_depth=10, n_estimators=1000)
xgb1.fit(XXX_train_part, yyy_train_part)
xgb_valid_pred = xgb1.predict_proba(XXX_valid)[:, 1]

roc_auc_score(yyy_valid, xgb_valid_pred)

0.73012406445701017

In [71]:
xgb1 = XGBClassifier(learning_rate=0.1, max_depth = 10, n_estimators=500)
xgb1.fit(XXX_train_part, yyy_train_part)
xgb_valid_pred = xgb1.predict_proba(XXX_valid)[:, 1]

roc_auc_score(yyy_valid, xgb_valid_pred)

0.71802782614183347

In [70]:
xgb1 = XGBClassifier(learning_rate=0.05, max_depth=10, n_estimators=1000)
xgb1.fit(XXX_train_part, yyy_train_part)
xgb_valid_pred = xgb1.predict_proba(XXX_valid)[:, 1]

roc_auc_score(yyy_valid, xgb_valid_pred)

0.73016979999858544

In [56]:
xgb1 = XGBClassifier(learning_rate=0.1, max_depth=10, n_estimators=500)
xgb1.fit(XXX_train, y_train)


XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=10,
       min_child_weight=1, missing=None, n_estimators=500, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [57]:
xgb1_pred = xgb1.predict_proba(XXX_test)[:, 1]

pd.Series(xgb1_pred, name='dep_delayed_15min').to_csv('D:/xgb_manyfeats.csv', index_label='id', header=True)

In [58]:
mixedpred = logit_test_pred*0.3 + xgb1_pred*0.7

In [59]:
pd.Series(mixedpred, name='dep_delayed_15min').to_csv('D:/mixed.csv', index_label='id', header=True)

In [62]:
mixedpred2 = logit_test_pred*0.35 + xgb1_pred*0.65

In [63]:
pd.Series(mixedpred2, name='dep_delayed_15min').to_csv('D:/mixed2.csv', index_label='id', header=True)

In [64]:
mixedpred3 = logit_test_pred*0.2 + xgb1_pred*0.8

In [65]:
pd.Series(mixedpred3, name='dep_delayed_15min').to_csv('D:/mixed3.csv', index_label='id', header=True)

In [44]:
from sklearn.model_selection import GridSearchCV

In [53]:
paramslist = {'learning_rate': [0.1], 'max_depth': [5], 'min_child_weight': [1],
             'gamma': [0], 'subsample': [0.8], 'colsample_bytree': [0.8]}

In [54]:
#gr=GridSearchCV(xgb1, paramslist, scoring = 'roc_auc')

In [55]:
#gr.fit(XXX_train_part, yyy_train_part)

MemoryError: 

In [77]:
tab1 = pd.read_csv('D:/logit_manyfeats.csv', index_col = 'id')
tab2 = pd.read_csv('D:/xgb_manyfeats.csv', index_col = 'id')

In [78]:
tab1.head()


,dep_delayed_15min
id,
0,0.138151
1,0.075304
2,0.074411
3,0.148571
4,0.224951


In [107]:
tab3 = 0.31*tab1 + 0.69*tab2

In [108]:
tab3.mean(), tab1.mean(), tab2.mean()

(dep_delayed_15min    0.192454
 dtype: float64, dep_delayed_15min    0.193527
 dtype: float64, dep_delayed_15min    0.191972
 dtype: float64)

In [99]:
adiff = 0.191972 - 0.192438

In [102]:
adiff

-0.0004659999999999942

In [104]:
tab4 = tab2 - adiff

In [105]:
tab4.mean(), tab2.mean()

(dep_delayed_15min    0.192438
 dtype: float64, dep_delayed_15min    0.191972
 dtype: float64)

In [109]:
tab3.to_csv('D:/mixed8.csv', index_label='id', header=True)